In [ ]:
!pip uninstall opencv-python-headless -y 

!pip install opencv-python --upgrade

In [1]:
import os
import sys
import glob
import time
import math
import cv2
import numpy as np
from tqdm import tqdm
import pandas as pd

COSINE_THRESHOLD = 0.5



In [2]:
def match(recognizer, feature1, dictionary):
    max_score = 0.0
    sim_user_id = ""
    for user_id, feature2 in zip(dictionary.keys(), dictionary.values()):
        score = recognizer.match(
            feature1, feature2, cv2.FaceRecognizerSF_FR_COSINE)
        if score >= max_score:
            max_score = score
            sim_user_id = user_id
    if max_score < COSINE_THRESHOLD:
        return False, ("", 0.0)
    return True, (sim_user_id, max_score)

def recognize_face(image, face_detector, face_recognizer, file_name=None):
    channels = 1 if len(image.shape) == 2 else image.shape[2]
    if channels == 1:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    if channels == 4:
        image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)

    if image.shape[0] > 1000:
        image = cv2.resize(image, (0, 0),
                           fx=500 / image.shape[0], fy=500 / image.shape[0])

    height, width, _ = image.shape
    face_detector.setInputSize((width, height))
    try:
        dts = time.time()
        _, faces = face_detector.detect(image)
        if file_name is not None:
            assert len(faces) > 0, f'the file {file_name} has no face'

        faces = faces if faces is not None else []
        features = []
        print(f'time detection  = {time.time() - dts}')
        for face in faces:
            rts = time.time()

            aligned_face = face_recognizer.alignCrop(image, face)
            feat = face_recognizer.feature(aligned_face)
            print(f'time recognition  = {time.time() - rts}')

            features.append(feat)
        return features, faces
    except Exception as e:
        print(e)
        print(file_name)
        return None, None



In [4]:
def main():
    
    # contain npy for embedings and registration photos
    directory = 'data'

    # Init models face detection & recognition
    weights = os.path.join(directory, "models2",
                           "face_detection_yunet_2023mar_int8.onnx")
    face_detector = cv2.FaceDetectorYN_create(weights, "", (0, 0))
    face_detector.setScoreThreshold(0.87)

    weights = os.path.join(directory, "models2", "face_recognition_sface_2021dec_int8.onnx")
    face_recognizer = cv2.FaceRecognizerSF_create(weights, "")

    # Get registered photos and return as npy files
    # File name = id name, embeddings of a photo is the representative for the id
    # If many files have the same name, an average embedding is used
    dictionary = {}
    # the tuple of file types, please ADD MORE if you want
    types = ('*.jpg', '*.png', '*.jpeg', '*.JPG', '*.PNG', '*.JPEG')
    files = []
    for a_type in types:
        files.extend(glob.glob(os.path.join(directory, 'images', a_type)))

    files = list(set(files))

    for file in tqdm(files):
        image = cv2.imread(file)
        feats, faces = recognize_face(
            image, face_detector, face_recognizer, file)
        if faces is None:
            continue
        user_id = os.path.splitext(os.path.basename(file))[0]
        dictionary[user_id] = feats[0]
    
    print(f'there are {len(dictionary)} ids')
    print(pd.DataFrame(dictionary.keys()))
    capture = cv2.VideoCapture(0)
    
    # Get video properties
    frame_width = int(capture.get(3))
    frame_height = int(capture.get(4))
    fps = int(capture.get(cv2.CAP_PROP_FPS))
    
    if not capture.isOpened():
        sys.exit()

    while True:
        start_hand = time.time()
        result, image = capture.read()
        if result is False:
            cv2.waitKey(0)
            break

        fetures, faces = recognize_face(image, face_detector, face_recognizer)
        if faces is None:
            continue

        for idx, (face, feature) in enumerate(zip(faces, fetures)):
            result, user = match(face_recognizer, feature, dictionary)
            box = list(map(int, face[:4]))
            color = (0, 255, 0) if result else (0, 0, 255)
            thickness = 2
            cv2.rectangle(image, box, color, thickness, cv2.LINE_AA)

            id_name, score = user if result else (f"unknown_{idx}", 0.0)
            text = "{0} ({1:.2f})".format(id_name, score)
            position = (box[0], box[1] - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            scale = 0.6
            cv2.putText(image, text, position, font, scale,
                        color, thickness, cv2.LINE_AA)

        # Write the frame with rectangles and recognized names to the output video
        cv2.imshow("face recognition", image)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        end_hand = time.time()
        print(f'speed of a loop = {end_hand - start_hand} means {1/(end_hand - start_hand)} frames per second')

    capture.release()
    cv2.destroyAllWindows()
    return dictionary


if __name__ == '__main__':
    dictionary = main()

  4%|███▌                                                                               | 1/23 [00:00<00:03,  6.57it/s]

time detection  = 0.041544198989868164
time recognition  = 0.06402206420898438
time detection  = 0.031957149505615234


 13%|██████████▊                                                                        | 3/23 [00:00<00:03,  5.95it/s]

time recognition  = 0.07028579711914062
time detection  = 0.04786944389343262
time recognition  = 0.04000377655029297


 17%|██████████████▍                                                                    | 4/23 [00:00<00:03,  6.23it/s]

...

time detection  = 0.07785248756408691
time recognition  = 0.06537151336669922
speed of a loop = 0.16315841674804688 means 6.129012648757335 frames per second
time detection  = 0.06703066825866699
time recognition  = 0.06548404693603516
speed of a loop = 0.14397478103637695 means 6.945660849779921 frames per second
time detection  = 0.07041096687316895
time recognition  = 0.0740816593170166
speed of a loop = 0.15117669105529785 means 6.614776345608842 frames per second
time detection  = 0.08217501640319824
time recognition  = 0.06624746322631836
speed of a loop = 0.1585385799407959 means 6.307612950572892 frames per second
time detection  = 0.0704493522644043
time recognition  = 0.06916356086730957
speed of a loop = 0.1564924716949463 means 6.390083747602369 frames per second
time detection  = 0.08688545227050781
time recognition  = 0.08268237113952637
speed of a loop = 0.17691612243652344 means 5.652396097245432 frames per second
time detection  = 0.07930231094360352
time recognition  

time recognition  = 0.06570219993591309
speed of a loop = 0.15008807182312012 means 6.662754660333749 frames per second
time detection  = 0.07427811622619629
time recognition  = 0.06645894050598145
speed of a loop = 0.14993906021118164 means 6.669376202515544 frames per second
time detection  = 0.08034515380859375
time recognition  = 0.07510542869567871
speed of a loop = 0.1665191650390625 means 6.005314762210208 frames per second
time detection  = 0.06078004837036133
time recognition  = 0.08116483688354492
speed of a loop = 0.15373539924621582 means 6.504682752984198 frames per second
time detection  = 0.07318782806396484
time recognition  = 0.06336069107055664
speed of a loop = 0.14645171165466309 means 6.828189228392398 frames per second
time detection  = 0.08650755882263184
speed of a loop = 0.09201526641845703 means 10.867761828263461 frames per second
time detection  = 0.08178567886352539
speed of a loop = 0.10825204849243164 means 9.237700477048378 frames per second
time detecti

time detection  = 0.08403539657592773
time recognition  = 0.06554746627807617
speed of a loop = 0.15780949592590332 means 6.336754288027968 frames per second
time detection  = 0.08010721206665039
time recognition  = 0.0666346549987793
speed of a loop = 0.157700777053833 means 6.341122844680964 frames per second
time detection  = 0.07326006889343262
time recognition  = 0.0669558048248291
speed of a loop = 0.15050983428955078 means 6.644084120617662 frames per second
time detection  = 0.08234643936157227
time recognition  = 0.057970523834228516
speed of a loop = 0.1499652862548828 means 6.668209856915739 frames per second
time detection  = 0.07583856582641602
time recognition  = 0.0603795051574707
speed of a loop = 0.15633153915405273 means 6.396661898240359 frames per second
time detection  = 0.08352875709533691
time recognition  = 0.06271696090698242
speed of a loop = 0.14960789680480957 means 6.684139148782233 frames per second
time detection  = 0.07626962661743164
time recognition  =

time detection  = 0.08037781715393066
time recognition  = 0.06664538383483887
speed of a loop = 0.1552438735961914 means 6.441478023159382 frames per second
time detection  = 0.07173895835876465
time recognition  = 0.06743144989013672
speed of a loop = 0.15654778480529785 means 6.38782593598321 frames per second
time detection  = 0.06998682022094727
time recognition  = 0.07915234565734863
speed of a loop = 0.15596866607666016 means 6.411544223302455 frames per second
time detection  = 0.08255219459533691
time recognition  = 0.06380653381347656
speed of a loop = 0.15381169319152832 means 6.501456288858266 frames per second
time detection  = 0.07233524322509766
time recognition  = 0.06870317459106445
speed of a loop = 0.15957951545715332 means 6.266468457027602 frames per second
time detection  = 0.07530832290649414
time recognition  = 0.06143331527709961
speed of a loop = 0.14126873016357422 means 7.0787073603769635 frames per second
time detection  = 0.08172011375427246
time recognitio

time recognition  = 0.0656585693359375
speed of a loop = 0.14936614036560059 means 6.694957756505722 frames per second
time detection  = 0.07923030853271484
time recognition  = 0.06607508659362793
speed of a loop = 0.14981889724731445 means 6.674725407631615 frames per second
time detection  = 0.07506966590881348
time recognition  = 0.0632021427154541
speed of a loop = 0.1506800651550293 means 6.636577963854316 frames per second
time detection  = 0.06886100769042969
time recognition  = 0.07994985580444336
speed of a loop = 0.17273592948913574 means 5.789183541359849 frames per second
time detection  = 0.08262252807617188
time recognition  = 0.05493497848510742
speed of a loop = 0.14316034317016602 means 6.985174650017154 frames per second
time detection  = 0.07768940925598145
time recognition  = 0.05695629119873047
speed of a loop = 0.16567039489746094 means 6.036081465363405 frames per second
time detection  = 0.07586526870727539
time recognition  = 0.06840205192565918
speed of a loop

In [ ]:
while True:
    start_hand = time.time()
    result, image = capture.read()
    if result is False:
        cv2.waitKey(0)
        break

    fetures, faces = recognize_face(image, face_detector, face_recognizer)
    if faces is None:
        continue

    for idx, (face, feature) in enumerate(zip(faces, fetures)):
        result, user = match(face_recognizer, feature, dictionary)
        box = list(map(int, face[:4]))
        color = (0, 255, 0) if result else (0, 0, 255)
        thickness = 2
        cv2.rectangle(image, box, color, thickness, cv2.LINE_AA)

        id_name, score = user if result else (f"unknown_{idx}", 0.0)
        text = "{0} ({1:.2f})".format(id_name, score)
        position = (box[0], box[1] - 10)
        font = cv2.FONT_HERSHEY_SIMPLEX
        scale = 0.6
        cv2.putText(image, text, position, font, scale,
                    color, thickness, cv2.LINE_AA)

    # Write the frame with rectangles and recognized names to the output video
    cv2.imshow("face recognition", image)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    end_hand = time.time()
    print(f'speed of a loop = {end_hand - start_hand} means {1/(end_hand - start_hand)} frames per second')

capture.release()
cv2.destroyAllWindows()
return dictionary

In [ ]:
print(dictionary['9213225'])

In [ ]:
def main():
    
    # contain npy for embedings and registration photos
    directory = 'data'

    # Init models face detection & recognition
    weights = os.path.join(directory, "models2",
                           "face_detection_yunet_2023mar_int8.onnx")
    face_detector = cv2.FaceDetectorYN_create(weights, "", (0, 0))
    face_detector.setScoreThreshold(0.87)

    weights = os.path.join(directory, "models2", "face_recognition_sface_2021dec_int8.onnx")
    face_recognizer = cv2.FaceRecognizerSF_create(weights, "")

    # Get registered photos and return as npy files
    # File name = id name, embeddings of a photo is the representative for the id
    # If many files have the same name, an average embedding is used
    dictionary = {}
    # the tuple of file types, please ADD MORE if you want
    types = ('*.jpg', '*.png', '*.jpeg', '*.JPG', '*.PNG', '*.JPEG')
    files = []
    for a_type in types:
        files.extend(glob.glob(os.path.join(directory, 'images', a_type)))

    files = list(set(files))

    for file in tqdm(files):
        image = cv2.imread(file)
        feats, faces = recognize_face(
            image, face_detector, face_recognizer, file)
        if faces is None:
            continue
        user_id = os.path.splitext(os.path.basename(file))[0]
        dictionary[user_id] = feats[0]

    print(f'there are {len(dictionary)} ids')
    capture = cv2.VideoCapture('Test_video.mov')
    
    # Get video properties
    frame_width = int(capture.get(3))
    frame_height = int(capture.get(4))
    fps = int(capture.get(cv2.CAP_PROP_FPS))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    output_video = cv2.VideoWriter("Output_Video.mp4", fourcc, fps, (frame_width, frame_height))

    
    
    
    
    if not capture.isOpened():
        sys.exit()

    while True:
        start_hand = time.time()
        result, image = capture.read()
        if result is False:
            cv2.waitKey(0)
            break

        fetures, faces = recognize_face(image, face_detector, face_recognizer)
        if faces is None:
            continue

        for idx, (face, feature) in enumerate(zip(faces, fetures)):
            result, user = match(face_recognizer, feature, dictionary)
            box = list(map(int, face[:4]))
            color = (0, 255, 0) if result else (0, 0, 255)
            thickness = 2
            cv2.rectangle(image, box, color, thickness, cv2.LINE_AA)

            id_name, score = user if result else (f"unknown_{idx}", 0.0)
            text = "{0} ({1:.2f})".format(id_name, score)
            position = (box[0], box[1] - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            scale = 0.6
            cv2.putText(image, text, position, font, scale,
                        color, thickness, cv2.LINE_AA)

        # Write the frame with rectangles and recognized names to the output video
        output_video.write(image)
        cv2.imshow("face recognition", image)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        end_hand = time.time()
        print(f'speed of a loop = {end_hand - start_hand} means {1/(end_hand - start_hand)} frames per second')

    capture.release()
    output_video.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    main()

In [ ]:
dictionary['Abdelrahman Sabry']

In [ ]:
import numpy as np
# Create a sample dictionary containing NumPy arrays
data_dict = {
    "array1": np.array([1, 2, 3]),
    "array2": np.array([4, 5, 6]),
    "array3": np.array([7, 8, 9])
}

In [ ]:
np.savez()

In [ ]:
pip list

In [ ]:
import pickle
file_name = "C:\\Users\\bedo1\\Downloads\\data.pkl"
with open(file_name, "rb") as file:  # Use 'rb' mode for reading binary data
    dictionary = pickle.load(file_name)

In [ ]:
pip list

In [4]:
pip list | grep fastapi


Note: you may need to restart the kernel to use updated packages.


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# ============================================================================
        fetures, faces = recognize_face(image)
        for idx, (face, feature) in enumerate(zip(faces, fetures)):
            result, user = match( feature)
            box = list(map(int, face[:4]))
            color = (0, 255, 0) if result else (0, 0, 255)
            thickness = 2
            cv2.rectangle(image, box, color, thickness, cv2.LINE_AA)

            id_name, score = user if result else (f"unknown_{idx}", 0.0)
            text = "{0} ({1:.2f})".format(id_name, score)
            position = (box[0], box[1] - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            scale = 0.6
            cv2.putText(image, text, position, font, scale,
                        color, thickness, cv2.LINE_AA)
            if text not in recognized_students:
                attended_students.append(text)
                recognized_students.append(text)


# ===========================================================================

In [20]:
# Import necessary libraries
import os
import time
import cv2
import pandas as pd
import numpy as np
import pickle
from datetime import date
from io import BytesIO
id_list = [9213225,9210380,9210352]
def get_attendance_list():
    df = pd.read_excel("all.xlsx")
    today = date.today()
    df[today] = 0

    for id in id_list:
        df.loc[df['ID'] == id,today] = 1

    # Create an in-memory binary stream to hold the Excel file content
    excel_stream = BytesIO()

    # Write the DataFrame to the in-memory Excel stream
    df.to_excel(excel_stream, index=False)  # The 'index=False' option removes the index column

    # Move the stream position to the beginning before returning the content
    excel_stream.seek(0)

    return excel_stream,df

In [21]:
excel,df = get_attendance_list()

In [22]:
df[df['ID'] == 9213225]

,Section,Name,الحالة,ID,رقم\nالجلوس,2023-08-03
87,2,عبد الرحمن صبري شعبان علي,مستجد,9213225,32090,1


In [10]:
df

,Section,Name,الحالة,ID,رقم\nالجلوس,2023-08-03
0,1,ابرام كمال عزيز كامل عبد الشهيد,مستجد,9210004,32001,0
1,1,ابراهيم خالد محمد عبد العزيز,مستجد,9210010,32002,0
2,1,احمد اسامه سعد ياسين,مستجد,9210023,32003,0
3,1,احمد اسامه فتحى فرج,مستجد,9210025,32004,0
4,1,احمد اشرف احمد عبد العزيز,مستجد,9210030,32005,0
...,...,...,...,...,...,...
199,4,يوسف محمد شعبان عبد الهادى,مستجد,9211433,32200,0
200,4,يوسف محمد عبدالوهاب محمد,مستجد,9211437,32201,0
201,4,يوسف محمد يوسف محمد ابراهيم,مستجد,9211444,32202,0
202,4,يوسف معتزبالله احمد الصاوى,مستجد,9211449,32203,0
